Justus Tulowiecki

Importing necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV , GridSearchCV
from sklearn.preprocessing import LabelEncoder, RobustScaler, MinMaxScaler, StandardScaler
from scipy.special import boxcox, boxcox1p
import scipy.stats as ss
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier,StackingClassifier
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as MSE
from sklearn.feature_selection import SelectKBest,chi2, RFE
import seaborn as sns
from lightgbm import LGBMClassifier

from IPython.core.interactiveshell import InteractiveShell #allows us to see multiple outputs without having to use the print function

Loading in the training and testing data. I did some minor exploration in excel since the data didn't have too many rows/features to see if there would be any issues when loading the data.

In [2]:
names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num','marital-status','occupation','relationship', 'race', 'sex', 'capital-gain:', 'capital-loss', 'hours-per-week-','native-country']

train = pd.read_csv('train-features.csv', names = names, na_values = ['?', ' ?', '? '])
test = pd.read_csv('test-features.csv', names = names, na_values = ['?', ' ?', '? '])
train_label = pd.read_csv('train-output.csv', names=['Income'])

train_len = len(train)
test_len = len(test)

X = pd.concat([train, test]) #combine train and test

Verifying that columns were read in correctly. Additionally, looking at column types and if there are any missing values.

In [3]:
train.info()
print('Total length of train and test is:', train_len+test_len)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age                32561 non-null int64
workclass          30725 non-null object
fnlwgt             32561 non-null int64
education          32561 non-null object
education-num      32561 non-null int64
marital-status     32561 non-null object
occupation         30718 non-null object
relationship       32561 non-null object
race               32561 non-null object
sex                32561 non-null object
capital-gain:      32561 non-null int64
capital-loss       32561 non-null int64
hours-per-week-    32561 non-null int64
native-country     31978 non-null object
dtypes: int64(6), object(8)
memory usage: 3.5+ MB
Total length of train and test is: 48842


Taking a look at the first few rows of the data.

In [4]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain:,capital-loss,hours-per-week-,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


Education number is categorical, so we must change the column type. Next, I will look at a simple summary of the data.

In [24]:
X['education-num'] = X['education-num'].astype('object')
round(X.describe(),2)

,age,capital-gain:,capital-loss,hours-per-week-,is_white,23/older,Full-Time,ednum-grtr-12,hrs-sq
count,48842.00,48842.00,48842.0,48842.00,48842.00,48842.00,48842.00,48842.00,48842.00
mean,38.64,1079.07,87.5,40.42,0.86,0.88,0.76,0.55,41.34
std,13.71,7452.02,403.0,12.39,0.35,0.33,0.43,0.50,12.43
min,17.00,0.00,0.0,1.00,0.00,0.00,0.00,0.00,0.00
25%,28.00,0.00,0.0,40.00,1.00,1.00,1.00,0.00,41.00
50%,37.00,0.00,0.0,40.00,1.00,1.00,1.00,1.00,42.00
75%,48.00,0.00,0.0,45.00,1.00,1.00,1.00,1.00,47.00
max,90.00,99999.00,4356.0,99.00,1.00,1.00,1.00,1.00,99.00


Using Pandas Profiling to perform a quick EDA. 

In [ ]:
datacs_profile = pandas_profiling.ProfileReport(X)
datacs_profile

Feature engineering:
The following section was updated iteratively to experiment with what features worked or not based off of the final model performance. Education was dropped as it was a duplicate field of the education-num feature. I filled the NA in native country with United States as it was the most common. Education was dropped as it is a duplicate feature.

I filled the native country with United States as it was the most common in the dataset. I decided to keep missing values in other features, like workclass and occupation as their own class of "Unknown". When it came to feature engineering, I tested various feature transformations but the following resulted in the best success. I dropped fnlwgt as it showed no correlation to our response variable.

In [6]:
X['native-country'].fillna('United-States', inplace=True)
X['workclass'].fillna('Unknown', inplace=True)
X['occupation'].fillna('Unknown', inplace=True)
X['is_white'] = X['race'].apply(lambda x: 1 if str.strip(x) == 'White' else 0)
X['23/older'] = X['age'].apply(lambda x: 1 if x > 22 else 0)
X['Full-Time'] = X['hours-per-week-'].apply(lambda x: 1 if x>39 else 0)
X['ednum-grtr-12'] = X['education-num'].apply(lambda x: 1 if x > 9 else 0)
X['hrs-sq'] = X['hours-per-week-'].apply(lambda x: x^2)
X.drop('education', axis='columns', inplace=True) #duplicate field for eduction-num
X.drop(['fnlwgt'], inplace=True, axis='columns')

This section gets dummy variables for the categorical variables. After variables have been encoded, a boxcox transformation is applied where ss.boxcox_normmax is used to find the optimal parameter to fit the feature to a normal distribution. Boxcox1p is used since data must be strictly greater than 0. After data is transformed, it is scaled.

In [28]:
X['workclass'].dtype

dtype('O')

In [ ]:
X = pd.get_dummies(X).reset_index(drop=True)


for i in range(X.shape[1]):
    if
    X.iloc[:,i] = boxcox1p(X.iloc[:,i], ss.boxcox_normmax(X.iloc[:,i]+1))

scaler = RobustScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)


In [ ]:
train_feat = X[:train_len]
print(train_feat.shape)

test_feat = X[train_len:]
print(test_feat.shape)

In [ ]:
xgbc = XGBClassifier(max_depth=5)
xgbc.fit(train_feat, train_label.values.ravel())
y_pred_xgb = xgbc.predict(test_feat)
y_pred_train = xgbc.predict(train_feat)

cvs_xgb = cross_val_score(xgbc,train_feat, train_label.values.ravel(), cv=5)

print('Cross-val-score for train data', cvs_xgb)
print('Average cross-val-score for train data', cvs_xgb.mean())

submit = pd.read_csv('submission.csv')
submit.iloc[:,-1] = y_pred_xgb
submit.to_csv('XGB_Submission.csv', index=False)

importances_xgb = pd.Series(data=xgbc.feature_importances_,
                        index= train_feat.columns)
importances_xgb = importances_xgb.sort_values(ascending=False)[:20]
sns.barplot(x=importances_xgb.index, y=importances_xgb)
plt.xticks(rotation=90)
plt.show()


Tuning actually resulted in worse scores and overfitting, so I kept with default model parameters. Other algorithms were able to achieve similar results, namely LightGBM and Adaboost. I tested many classification algorithms including random forest, naive bayes, and logistic regression but the results were much worse than the models that utilized boosting. XGBoost had the best cross-val-scores and the best score when submitted to kaggle. Additionally, I tried a stacking method which used LightGBM, Adaboost, and XGBoost. Interestingly this  resulted in a worse score than just using XGBoost alone.

In [ ]:
'''params_xgb = {
            'learning_rate': [ 0.01, 0.05, 0.1],
            'n_estimators': [1000, 1500, 3000],
            'max_depth' : [3, 4, 5, 7],
            'subsample': [0.7, 0.8, 0.9],
            'colsample_bytree': [0.4, 0.6, 0.8],
}

grid_xgb = GridSearchCV(estimator=xgbc,
                       param_grid=params_xgb,
                        #n_iter=5,
                       cv=5,
                        error_score='accuracy',
                        verbose=1,
                       n_jobs=5
                          )
'''                       
                    

In [ ]:
grid_xgb.fit(train_feat, train_label.values.ravel())

In [ ]:
best_hyperparams = grid_xgb.best_params_
print('Best hyperparameters:\n', best_hyperparams)